<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/catdog/AnalyzeCatDogResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r abstention
!pip uninstall abstention
!git clone https://github.com/blindauth/abstention.git
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

rm: cannot remove 'abstention': No such file or directory
Cloning into 'abstention'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 127 (delta 75), reused 91 (delta 39), pack-reused 0
Receiving objects: 100% (127/127), 65.16 KiB | 8.15 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-ajk5m_8n/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
Cloning into 'abstention_experiments'...
remote: Enumerating objects: 1125, done.
remote: Counting objects: 100% (1125/1125), done.
remote: Compressing objects: 100% (1120/1120), done.
remote: Total 4063 (delta 48), reused 1048 (delta 5), pack-reused 2938
Receiving objects: 100% (4063/4063), 289.56 MiB | 44.42 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Checking out files: 100% (3893/3893), done.


In [2]:
!ls abstention_experiments/imdb

AnalyzeIMDBandCatDogResults.ipynb  record_6_model_mgGta_preds
imdb_abstention_results.json.gz    record_7_model_Si0xy_preds
record_10_model_Vs2cs_preds	   record_8_model_yds3D_preds
record_1_model_FObIR_preds	   record_9_model_dCvxV_preds
record_2_model_T4HRC_preds	   Run_IMDB_Experiments.ipynb
record_3_model_ZGX50_preds	   test_labels.txt.gz
record_4_model_hWBrJ_preds	   valid_labels.txt.gz
record_5_model_mBWKc_preds


In [0]:
import gzip
import json
imdb_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/imdb/imdb_abstention_results.json.gz").read())
catdog_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/catdog/catdog_abstention_results.json.gz").read())

In [4]:
from abstention.figure_making_utils import (
    wilcox_srs, get_ustats_mat,
    get_tied_top_and_worst_methods)
import numpy as np
from collections import defaultdict

abstfracs = ['0.15', '0.3']

abstfrac_to_methodnames = {
    '0.15': ['distfrom0.5:None',
             'distfrom0.5:EM',
             'js:None',
             'js:EM',
             'mcwindowdelta_tpratfpr_15pc:None',
             'mcwindowdelta_tpratfpr_15pc:EM',
             'fumera_tpratfpr_15pc:None',
             'fumera_tpratfpr_15pc:EM',
             'uncertainty:None',
             'uncertainty:EM'],
    '0.3': ['distfrom0.5:None',
            'distfrom0.5:EM',
            'js:None',
            'js:EM',
            'mcwindowdelta_tpratfpr_30pc:None',
            'mcwindowdelta_tpratfpr_30pc:EM',
            'fumera_tpratfpr_30pc:None',
            'fumera_tpratfpr_30pc:EM',
            'uncertainty:None',
            'uncertainty:EM']
}

nicemethodname_to_adapt_to_abstfrac_to_methodname = {
    "Est $\Delta$Metric (Ours)": {
        'yesadapt': {
            '0.3': 'mcwindowdelta_tpratfpr_30pc:EM',
            '0.15': 'mcwindowdelta_tpratfpr_15pc:EM'
        }, 'noadapt': {
            '0.3': 'mcwindowdelta_tpratfpr_30pc:None',
            '0.15': 'mcwindowdelta_tpratfpr_15pc:None'
        }
    },
    "JS Div. (Ours)": {
        'yesadapt': {'0.3': 'js:EM', '0.15': 'js:EM'},
        'noadapt': {'0.3': 'js:None', '0.15': 'js:None'}
    },
    "Dist. from 0.5": {
        'yesadapt': {'0.3': 'distfrom0.5:EM', '0.15': 'distfrom0.5:EM'},
        'noadapt': {'0.3': 'distfrom0.5:None', '0.15': 'distfrom0.5:None'}
    },
    "Test-time Dropout": {
        'yesadapt': {'0.3': 'uncertainty:EM', '0.15': 'uncertainty:EM'},
        'noadapt': {'0.3': 'uncertainty:None', '0.15': 'uncertainty:None'}
    },
    "Fumera et al.": {
        'yesadapt': {
              '0.3': 'fumera_tpratfpr_30pc:EM',
              '0.15': 'fumera_tpratfpr_15pc:EM'
        },
        'noadapt': {
              '0.3': 'fumera_tpratfpr_30pc:None',
              '0.15': 'fumera_tpratfpr_15pc:None'
        }
    }
}
nicemethodname_orders = [
    "Est $\Delta$Metric (Ours)", "JS Div. (Ours)",
    "Dist. from 0.5", "Fumera et al.", "Test-time Dropout"]

def stdderr(vals):
  return np.std(vals, ddof=1)/np.sqrt(len(vals))

#identify the best methods
datasetname_to_abstfrac_to_bestmethods = defaultdict(dict)  
for datasetname,dataset_results in [('imdb', imdb_abstention_results),
                                    ('catdog', catdog_abstention_results)]:
  for abstfrac in abstfracs:
    methodnames = abstfrac_to_methodnames[abstfrac]
    nsamples = len(list(
      dataset_results['metricname_to_abstfrac_to_abstnadapt_to_perf']
                        ['tpratfpr_0.01'][abstfrac].values())[0])
    ustats_mat = get_ustats_mat(
        method_to_perfs=
          dataset_results['metricname_to_abstfrac_to_abstnadapt_to_perf']
              ['tpratfpr_0.01'][abstfrac],
        method_names=methodnames,
        max_ustat=nsamples*(nsamples+1)/2 )
    tied_top_methods, tied_worst_methods =(
        get_tied_top_and_worst_methods(
            ustats_mat,
            methodnames,
            #0.05 threshold for one-sided test when N=30 is 159,
            #0.025 threshold is 137
            #0.005 threshold is 109
            #from https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
            threshold=137
        ))
    tied_top_methods = [methodnames[x] for x in tied_top_methods]
    print("bestmethods",datasetname,'tpratfpr_0.01', abstfrac, tied_top_methods)
    datasetname_to_abstfrac_to_bestmethods[datasetname][abstfrac] =\
      set(tied_top_methods)
    
    
thetable = """
\\begin{table*}[!h]
\\tiny
\\begin{center}
\\begin{tabular}{ | c c | c c c | c c c | }
\\hline
\\multicolumn{2}{| c |}{ } & \\multicolumn{3}{ c |}{IMDB Sensitivity @ 99\\% Specificity} & \\multicolumn{3}{c | }{Kaggle Cat v Dog Sensitivity @ 99\\% Specificity}\\\\
\\multicolumn{1}{| c}{Method} & \\multicolumn{1}{c |}{Adapted?} & Base & @30\\% Abst. & @15\\% Abst. & Base & @30\\% Abst. & @15\\% Abst.\\\\ \\hline
"""

for nicemethodname in nicemethodname_orders:
  for adapt in ['yesadapt', 'noadapt']:
    if adapt in nicemethodname_to_adapt_to_abstfrac_to_methodname[nicemethodname]:
      thetable += nicemethodname
      thetable += " & "+("Y" if adapt=='yesadapt' else 'N')
      for datasetname,dataset_results in [
          ('imdb', imdb_abstention_results),
          ('catdog', catdog_abstention_results)]:
        metricname_to_adaptname_to_baselineperfs =\
          dataset_results['metricname_to_adaptname_to_baselineperfs']
        metricname_to_abstfrac_to_abstnadapt_to_perf =\
          dataset_results['metricname_to_abstfrac_to_abstnadapt_to_perf']
        baseperfs = metricname_to_adaptname_to_baselineperfs['tpratfpr_0.01'][
                     ('EM' if adapt=='yesadapt' else 'None')]
        thetable += (" & "+str(np.round(np.mean(baseperfs),4))
                     +"$\\pm$"+str(np.round(stdderr(baseperfs),4)))
        for abstfrac in ['0.3', '0.15']:
          methodname =\
           nicemethodname_to_adapt_to_abstfrac_to_methodname[
            nicemethodname][adapt][abstfrac]
          isbest = (methodname in
            datasetname_to_abstfrac_to_bestmethods[datasetname][abstfrac])
          methodperfs = metricname_to_abstfrac_to_abstnadapt_to_perf[
            'tpratfpr_0.01'][abstfrac][methodname]
          thetable += " & "
          if (isbest):
            thetable += "\\textbf{"
          thetable += (str(np.round(np.mean(methodperfs),4))
                       +"$\pm$"+str(np.round(stdderr(methodperfs),4)))
          if (isbest):
            thetable += "}"
      thetable += "\\\\ \\hline\n"
      if (nicemethodname=="JS Div. (Ours)" and adapt=='noadapt'):
          thetable += "\\hline\n"

thetable += """\\end{tabular}
\\end{center}
\\caption{\\textbf{}. }
\\label{tab:imdb_and_catdog}
\\end{table*}
"""
print("\n\n"+thetable)

bestmethods imdb tpratfpr_0.01 0.15 ['mcwindowdelta_tpratfpr_15pc:EM']
bestmethods imdb tpratfpr_0.01 0.3 ['mcwindowdelta_tpratfpr_30pc:EM']
bestmethods catdog tpratfpr_0.01 0.15 ['mcwindowdelta_tpratfpr_15pc:EM']
bestmethods catdog tpratfpr_0.01 0.3 ['mcwindowdelta_tpratfpr_30pc:EM', 'fumera_tpratfpr_30pc:None']



\begin{table*}[!h]
\tiny
\begin{center}
\begin{tabular}{ | c c | c c c | c c c | }
\hline
\multicolumn{2}{| c |}{ } & \multicolumn{3}{ c |}{IMDB Sensitivity @ 99\% Specificity} & \multicolumn{3}{c | }{Kaggle Cat v Dog Sensitivity @ 99\% Specificity}\\
\multicolumn{1}{| c}{Method} & \multicolumn{1}{c |}{Adapted?} & Base & @30\% Abst. & @15\% Abst. & Base & @30\% Abst. & @15\% Abst.\\ \hline
Est $\Delta$Metric (Ours) & Y & 0.3571$\pm$0.004 & \textbf{0.7067$\pm$0.0067} & \textbf{0.5243$\pm$0.0059} & 0.3318$\pm$0.0061 & \textbf{0.6135$\pm$0.0114} & \textbf{0.4598$\pm$0.0082}\\ \hline
Est $\Delta$Metric (Ours) & N & 0.3571$\pm$0.004 & 0.6227$\pm$0.0082 & 0.4959$\pm$0.0057 & 0.33